In [3]:
using GynC
blas_set_num_threads(1)

@time ss = [sample(Config(Lausanne(i), relprop = 0.00001), 100) for i = 1:5]

  5.457211 seconds (4.75 M allocations: 514.021 MB, 3.78% gc time)


5-element Array{Union{Array{Float64,2},GynC.Sampling},1}:
 Sampling
  samples: (100,115)
  uniques: 19
  config:  Config:
 patient: 
 sigma:   0.1
 relprop: 1.0e-5
 thin:    1
 init:    4858380976513133059
 prior:   3655396727605176177
 Sampling
  samples: (100,115)
  uniques: 21
  config:  Config:
 patient: 
 sigma:   0.1
 relprop: 1.0e-5
 thin:    1
 init:    4858380976513133059
 prior:   3655396727605176177
 Sampling
  samples: (100,115)
  uniques: 15
  config:  Config:
 patient: 
 sigma:   0.1
 relprop: 1.0e-5
 thin:    1
 init:    4858380976513133059
 prior:   3655396727605176177
 Sampling
  samples: (100,115)
  uniques: 28
  config:  Config:
 patient: 
 sigma:   0.1
 relprop: 1.0e-5
 thin:    1
 init:    4858380976513133059
 prior:   3655396727605176177
 Sampling
  samples: (100,115)
  uniques: 27
  config:  Config:
 patient: 
 sigma:   0.1
 relprop: 1.0e-5
 thin:    1
 init:    4858380976513133059
 prior:   3655396727605176177

(:lausanne,1)(:lausanne,2)(:lausanne,3)(:lausanne,4)(:lausanne,5)(:lausanne,1)(:lausanne,1)(:lausanne,2)(:lausanne,2)(:lausanne,3)(:lausanne,3)(:lausanne,4)(:lausanne,4)(:lausanne,5)(:lausanne,5)

In [5]:
w = WeightedChain(ss...)

emiteration!(w)

GynC.WeightedChain(500x115 Array{Float64,2}:
 7309.9   7309.9   192.2    2.371    …  0.0009368    3.3169e-5   2.9879e-5 
 7309.79  7309.93  192.199  2.37098     0.000936799  3.31691e-5  2.9879e-5 
 7309.79  7309.93  192.199  2.37098     0.000936799  3.31691e-5  2.9879e-5 
 7309.79  7309.93  192.199  2.37098     0.000936799  3.31691e-5  2.9879e-5 
 7309.79  7309.93  192.199  2.37098     0.000936799  3.31691e-5  2.9879e-5 
 7309.8   7309.94  192.197  2.37096  …  0.000936804  3.3169e-5   2.98789e-5
 7309.63  7309.94  192.196  2.37094     0.000936806  3.31692e-5  2.98791e-5
 7309.63  7309.94  192.193  2.37094     0.000936804  3.31699e-5  2.98795e-5
 7309.63  7309.94  192.193  2.37094     0.000936804  3.31699e-5  2.98795e-5
 7309.66  7309.91  192.194  2.37095     0.000936801  3.31701e-5  2.98793e-5
 7309.66  7309.91  192.194  2.37095  …  0.000936801  3.31701e-5  2.98793e-5
 7309.66  7309.91  192.194  2.37095     0.000936801  3.31701e-5  2.98793e-5
 7309.66  7309.91  192.194  2.37095     0.0